<h1>Regression</h1>

<h2>Import Libaries</h2>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.stats.diagnostic as dg
from statsmodels.stats.diagnostic import het_white

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

<h2>Laden der Datensätze</h2>

In [2]:
#Load German Sentiment CSV file.
df_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Sentiment_Analysis/Dataset_DE_Bert_Vader.csv", sep=";",
                parse_dates=["created_at"])

#Change the date field
df_de["Date"] = pd.to_datetime(df_de.created_at).apply(lambda x: x.date())

#Change the values of sentiment bert field and add a new field for a numeric value sentiment value 
df_de["sentiment_bert"] = df_de["sentiment_bert"].replace(["neutral", "negative", "positive"], ["NEU", "NEG", "POS"])
df_de["sentiment_bert_value"] = df_de["sentiment_bert"].replace({"NEU": 0, "NEG": -1, "POS": 1})

#Create a new field for a numeric sentiment value
df_de["sentiment_vader_value"] = [1 if i >= 0.05 else -1 if i <= -0.05 else 0 for i in df_de["sentiment_vader"]]

#Create a df for adding the year and week to the df
df_de_week_year = pd.DataFrame([(i.isocalendar().week, i.isocalendar().year, i.isocalendar().weekday) for i in df_de["Date"]],
                               columns=["Week", "Year", "Weekday"])

#Recalulate the week
for each in range(len(df_de_week_year)):
    if ((df_de_week_year.iloc[each]["Weekday"] > 5) & (df_de_week_year.iloc[each]["Week"] != 1)):
        df_de_week_year.iloc[each]["Week"] = df_de_week_year.iloc[each]["Week"] - 1
    elif ((df_de_week_year.iloc[each]["Weekday"] > 5) & (df_de_week_year.iloc[each]["Week"] == 1)):
        df_de_week_year.iloc[each]["Week"] = 52

#Adding the week and year information to the df
df_de[["Week", "Year"]] = df_de_week_year[["Week", "Year"]]

tweet_id_friday_list = []
for each in range(len(df_de)):
    if (df_de["Date"][each].isocalendar().year == 2016) & (df_de["Date"][each].isocalendar().week == 12) & (df_de["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)
    elif (df_de["Date"][each].isocalendar().year == 2017) & (df_de["Date"][each].isocalendar().week == 15) & (df_de["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)
    elif (df_de["Date"][each].isocalendar().year == 2018) & (df_de["Date"][each].isocalendar().week == 13) & (df_de["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)
    elif (df_de["Date"][each].isocalendar().year == 2019) & (df_de["Date"][each].isocalendar().week == 16) & (df_de["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)
    elif (df_de["Date"][each].isocalendar().year == 2020) & (df_de["Date"][each].isocalendar().week == 15) & (df_de["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)
    elif (df_de["Date"][each].isocalendar().year == 2020) & (df_de["Date"][each].isocalendar().week == 18) & (df_de["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)

for each in tweet_id_friday_list:
    df_de["Week"][each] = df_de["Week"][each] + 1

/var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/ipykernel_20909/2487132561.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_de["Week"][each] = df_de["Week"][each] + 1


In [3]:
#Load English Sentiment CSV file.
df_en = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Sentiment_Analysis/Dataset_Bert_Vader.csv", sep=";",
                parse_dates=["created_at"])

#Change the date field
df_en["Date"] = pd.to_datetime(df_en.created_at).apply(lambda x: x.date())

#Add a new column for a numeric sentiment bert value
df_en["sentiment_bert_value"] = df_en["sentiment_bert"].replace({"NEU": 0, "NEG": -1, "POS": 1})

#Add a new column for a numeric sentiment vader value
df_en["sentiment_vader_value"] = [1 if i >= 0.05 else -1 if i <= -0.05 else 0 for i in df_en["sentiment_vader"]]

#Create a df for adding the year and week to the df
df_en_week_year = pd.DataFrame([(i.isocalendar().week, i.isocalendar().year, i.isocalendar().weekday) for i in df_en["Date"]],
                               columns=["Week", "Year", "Weekday"])

#Recalulate the week
for each in range(len(df_en_week_year)):
    if ((df_en_week_year.iloc[each]["Weekday"] > 5) & (df_en_week_year.iloc[each]["Week"] != 1)):
        df_en_week_year.iloc[each]["Week"] = df_en_week_year.iloc[each]["Week"] - 1
    elif ((df_en_week_year.iloc[each]["Weekday"] > 5) & (df_en_week_year.iloc[each]["Week"] == 1)):
        df_en_week_year.iloc[each]["Week"] = 52
        
#Adding the week and year information to the df
df_en[["Week", "Year"]] = df_en_week_year[["Week", "Year"]]
        
tweet_id_friday_list = []
for each in range(len(df_en)):
    if (df_en["Date"][each].isocalendar().year == 2016) & (df_en["Date"][each].isocalendar().week == 12) & (df_en["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)
    elif (df_en["Date"][each].isocalendar().year == 2017) & (df_en["Date"][each].isocalendar().week == 15) & (df_en["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)
    elif (df_en["Date"][each].isocalendar().year == 2018) & (df_en["Date"][each].isocalendar().week == 13) & (df_en["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)
    elif (df_en["Date"][each].isocalendar().year == 2019) & (df_en["Date"][each].isocalendar().week == 16) & (df_en["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)
    elif (df_en["Date"][each].isocalendar().year == 2020) & (df_en["Date"][each].isocalendar().week == 15) & (df_en["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)
    elif (df_en["Date"][each].isocalendar().year == 2020) & (df_en["Date"][each].isocalendar().week == 18) & (df_en["Date"][each].isocalendar().weekday == 5):
        tweet_id_friday_list.append(each)

for each in tweet_id_friday_list:
    df_en["Week"][each] = df_en["Week"][each] + 1

/var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/ipykernel_20909/1593085834.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en["Week"][each] = df_en["Week"][each] + 1


In [4]:
#Combine the different DFs
df = pd.concat([df_en, df_de]).reset_index(drop=True)

In [5]:
#Change the values from sentiment_bert_value from text to a num value
df["sentiment_bert_value"] = df["sentiment_bert"].replace({"NEU": 0, "NEG": -1, "POS": 1})

#Create a df for adding the year and week to the df
df_week_year = pd.DataFrame([(i.isocalendar().week, i.isocalendar().year) for i in df["Date"]], columns=["Week", "Year"])
#Adding the week and year information to the df
df[["Week", "Year"]] = df_week_year[["Week", "Year"]]

In [6]:
#Load Wirecard finance CSV file.
df_wdi = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/WDI.HM-2.csv", parse_dates=["Date"])
df_wdi = df_wdi.rename(columns={"Adj Close": "Adj_Close"})

#Load Euro Stoxx 50 CSV file.
df_stoxx = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/^STOXX50E.csv", parse_dates=["Date"])
#df_stoxx = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/^GDAXI.csv", parse_dates=["Date"])
df_stoxx = df_stoxx.rename(columns={"Adj Close": "Adj_Close"})

#Load Visa finance CSV file.
df_visa = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/V.csv", parse_dates=["Date"])
df_visa = df_visa.rename(columns={"Adj Close": "Adj_Close"})

#Load Master Card finance CSV file.
df_ma = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/MA.csv", parse_dates=["Date"])
df_ma = df_ma.rename(columns={"Adj Close": "Adj_Close"})

<h2>Data Preparation</h2>

In [7]:
#Change Date für df_wdi & df
df_wdi["Date"] = pd.to_datetime(df_wdi.Date).apply(lambda x: x.date())
df_stoxx["Date"] = pd.to_datetime(df_stoxx.Date).apply(lambda x: x.date())
df_visa["Date"] = pd.to_datetime(df_visa.Date).apply(lambda x: x.date())
df_ma["Date"] = pd.to_datetime(df_ma.Date).apply(lambda x: x.date())

In [8]:
def create_finance_df(df):
    #Create df for returning
    df_week = pd.DataFrame(columns=["Date", "Week", "Year", "Adj_Close"])
    
    #Start Week
    week = 5
    volume = 0
    
    #To loop through all lines in df
    for each in range(len(df)):
        #Get the date
        each_date = df["Date"][each]
        adj_close = df["Adj_Close"][each]
        
        #Get the Volume of all transactions in a week
        volume = volume + df["Volume"][each]

        #Check if the date is the last date
        if df["Date"][each] == datetime.date(2020,7,31):
            #Get Week Number, start and end day
            week = each_date.isocalendar().week
            year = each_date.isocalendar().year
            end_day = df["Date"].iloc[-1]
            
            #Add data to df
            new_row = pd.DataFrame([[each_date, week, year, adj_close, volume]],
                                   columns=["Date", "Week", "Year", "Adj_Close", "Volume"])
            df_week = pd.concat([df_week, new_row], ignore_index=True)
        else:
            #Get the next week number for checking
            next_week = df["Date"][each+1].isocalendar().week

            #Check if the next day is in another week 
            if each_date.isocalendar().week != next_week:
                #Get week number, year and last date
                week = each_date.isocalendar().week
                year = each_date.isocalendar().year
                end_day = each_date
                
                #Add data to df
                new_row = pd.DataFrame([[each_date, week, year, adj_close, volume]],
                                       columns=["Date", "Week", "Year", "Adj_Close", "Volume"])
                df_week = pd.concat([df_week, new_row], ignore_index=True)
                
                #Set Volume back to 0
                volume = 0
    
    #Add the yield from one week to another
    df_week["returns_week"] = df_week.Adj_Close.pct_change()+1
    #df_week["returns_week"] = np.log(df_week.Adj_Close.pct_change()+1)
    #df_week["returns_week"] = df_week.Adj_Close.pct_change()
    #df_week["returns_week_log"] = pd.Series(np.log(np.float64(df_week.Adj_Close))).pct_change()
    

    
    #Return the new DF
    return df_week

In [9]:
def concat_finance_df(df_wdi = df_wdi, df_stoxx = df_stoxx, df_ma = df_ma, df_v = df_visa):
    
    #Create week dfs for all financial dfs
    df_wdi_week = create_finance_df(df_wdi)
    df_stoxx_week = create_finance_df(df_stoxx)
    df_ma_week = create_finance_df(df_ma)
    df_v_week = create_finance_df(df_v)
    
    #combine all dfs
    df_week = pd.concat([df_wdi_week, df_stoxx_week.rename(columns={"returns_week": "returns_stoxx"})["returns_stoxx"]],
                        axis=1)
    df_week = pd.concat([df_week, df_ma_week.rename(columns={"returns_week": "returns_ma"})["returns_ma"]], axis=1)
    df_week = pd.concat([df_week, df_v_week.rename(columns={"returns_week": "returns_visa"})["returns_visa"]], axis=1)
    return df_week

In [10]:
def create_final_df(df_week, df, meta_data = False):
    
    #If meta data like Retweets should be used, this will calculate it.
    if meta_data:
        df["sentiment_bert_value"] = df["sentiment_bert_value"] * ((df["retweet_count"] + 1))
        df["sentiment_vader_value"] = df["sentiment_vader_value"] * ((df["retweet_count"] + 1))
    
    df_sum_year_week_bert = pd.DataFrame(df[["Week", "Year", "sentiment_bert_value"]].
                                    groupby(by=["Year", "Week"]).sum()).reset_index()

    df_sum_year_week_vader = pd.DataFrame(df[["Week", "Year", "sentiment_vader_value"]].
                                    groupby(by=["Year", "Week"]).sum()).reset_index()
    
    df_count_tweets_year_week = pd.DataFrame(df[["Week", "Year", "conversation_id"]].
                                    groupby(by=["Year", "Week"]).
             count()).reset_index().rename(columns={"conversation_id": "count_all_tweets"})
    
    df_week = df_week.merge(df_sum_year_week_bert)
    df_week = df_week.merge(df_sum_year_week_vader)
    df_week = df_week.merge(df_count_tweets_year_week)
    
    return df_week

In [11]:
def create_df(df_twitter = df.copy()):
    df_week = concat_finance_df()
    df_week_reg = create_final_df(df_week, df_twitter)
    
    df_week_reg["dif_sentiment_bert_value"] = df_week_reg["sentiment_bert_value"].diff()
    df_week_reg["dif_sentiment_vader_value"] = df_week_reg["sentiment_vader_value"].diff()

    df_week_reg.dropna(inplace=True)
    df_week_reg = df_week_reg.reset_index(drop=True)

    return df_week_reg

<h2>Linear Regression</h2>

In [13]:
def regression(df, regressor, output):
    
    x = df[regressor]
    y = df[output]
    
    x = sm.add_constant(x)

    model = sm.OLS(y,x).fit()
    
    predictions = model.predict(x)
    
    pritn_model = model.summary()
    
    print(pritn_model)
    
    return predictions, model

In [14]:
def WLS_regression(df, regressor, output, weiter):
    
    x = df[regressor]
    y = df[output]
    
    x = sm.add_constant(x)

    model = sm.WLS(y,x, weiter).fit()
    
    predictions = model.predict(x)
    
    pritn_model = model.summary()
    
    print(pritn_model)
    
    return predictions, model

<h2>Execute</h2>

<p>Multiple Lineare Regression der deutschen Tweets mit BERT</p>

In [88]:
regressor = ["dif_sentiment_bert_value", "returns_stoxx", "returns_ma", "returns_visa", "Volume", "count_all_tweets"]

regressand = ["returns_week"]

df_week_reg = create_df(df_de)

reg_predictions, reg_model = regression(df_week_reg, regressor, regressand)

                            OLS Regression Results                            
Dep. Variable:           returns_week   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.720
Method:                 Least Squares   F-statistic:                     100.8
Date:                Wed, 07 Dec 2022   Prob (F-statistic):           3.55e-61
Time:                        22:03:44   Log-Likelihood:                 290.96
No. Observations:                 234   AIC:                            -567.9
Df Residuals:                     227   BIC:                            -543.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [89]:
reg_model.pvalues

const                       5.863983e-01
dif_sentiment_bert_value    6.241022e-22
returns_stoxx               2.493319e-02
returns_ma                  6.897633e-01
returns_visa                4.056692e-01
Volume                      4.010908e-01
count_all_tweets            6.248443e-03
dtype: float64

In [90]:
reset = dg.linear_reset(reg_model, power=3, test_type="fitted", use_f=True)
reset.pvalue

/Users/tobias/Dev/FOM/Master_Thesis/master_thesis_3.9.10/.venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


2.4288902152711113e-23

<p>________________________________________________________________________________</p>
<p>Multiple lineare Regression in deutsch mit VADER</p>

In [91]:
regressor = ["dif_sentiment_vader_value", "returns_stoxx", "returns_ma", "returns_visa", "Volume", "count_all_tweets"]

regressand = ["returns_week"]

df_week_reg = create_df(df_de)

reg_predictions, reg_model = regression(df_week_reg, regressor, regressand)

                            OLS Regression Results                            
Dep. Variable:           returns_week   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     64.93
Date:                Wed, 07 Dec 2022   Prob (F-statistic):           1.49e-46
Time:                        22:04:14   Log-Likelihood:                 255.97
No. Observations:                 234   AIC:                            -497.9
Df Residuals:                     227   BIC:                            -473.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [92]:
reg_model.pvalues

const                        9.468218e-01
dif_sentiment_vader_value    6.043831e-07
returns_stoxx                1.089218e-02
returns_ma                   2.786104e-01
returns_visa                 9.983902e-01
Volume                       3.566145e-35
count_all_tweets             1.597010e-42
dtype: float64

In [93]:
reset = dg.linear_reset(reg_model, power=3, test_type="fitted", use_f=True)
reset.pvalue

/Users/tobias/Dev/FOM/Master_Thesis/master_thesis_3.9.10/.venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


3.8191544307145805e-37

<p>________________________________________________________________________________</p>
<p>Multiple lineare Regression mit BERT auf englischen Tweets</p>

In [97]:
regressor = ["dif_sentiment_bert_value", "returns_stoxx", "returns_ma", "returns_visa", "Volume", "count_all_tweets"]

regressand = ["returns_week"]

df_week_reg = create_df(df_en)

reg_predictions, reg_model = regression(df_week_reg, regressor, regressand)

                            OLS Regression Results                            
Dep. Variable:           returns_week   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     88.24
Date:                Wed, 07 Dec 2022   Prob (F-statistic):           1.52e-56
Time:                        22:05:28   Log-Likelihood:                 279.89
No. Observations:                 234   AIC:                            -545.8
Df Residuals:                     227   BIC:                            -521.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [98]:
reg_model.pvalues

const                       7.105238e-01
dif_sentiment_bert_value    8.725965e-10
returns_stoxx               1.431993e-02
returns_ma                  7.615167e-01
returns_visa                4.523541e-01
Volume                      1.291670e-02
count_all_tweets            7.274211e-04
dtype: float64

In [99]:
reset = dg.linear_reset(reg_model, power=3, test_type="fitted", use_f=True)
reset.pvalue

/Users/tobias/Dev/FOM/Master_Thesis/master_thesis_3.9.10/.venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


2.125824428392394e-24

<p>________________________________________________________________________________</p>
<p>Multiple lineare Regression mit VADER auf englische Tweets</p>

In [100]:
regressor = ["dif_sentiment_vader_value", "returns_stoxx", "returns_ma", "returns_visa", "Volume", "count_all_tweets"]

regressand = ["returns_week"]

df_week_reg = create_df(df_en)

reg_predictions, reg_model = regression(df_week_reg, regressor, regressand)

                            OLS Regression Results                            
Dep. Variable:           returns_week   R-squared:                       0.726
Model:                            OLS   Adj. R-squared:                  0.719
Method:                 Least Squares   F-statistic:                     100.2
Date:                Wed, 07 Dec 2022   Prob (F-statistic):           5.54e-61
Time:                        22:05:33   Log-Likelihood:                 290.50
No. Observations:                 234   AIC:                            -567.0
Df Residuals:                     227   BIC:                            -542.8
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [101]:
reg_model.pvalues

const                        3.653728e-01
dif_sentiment_vader_value    2.458160e-14
returns_stoxx                3.165560e-02
returns_ma                   5.609230e-01
returns_visa                 5.991945e-01
Volume                       5.982656e-01
count_all_tweets             2.556760e-01
dtype: float64

In [102]:
reset = dg.linear_reset(reg_model, power=3, test_type="fitted", use_f=True)
reset.pvalue

/Users/tobias/Dev/FOM/Master_Thesis/master_thesis_3.9.10/.venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


6.631405841830008e-20

<p>________________________________________________________________________________</p>
<p>Multiple lineare Regression mit BERT auf alle Tweets</p>

In [103]:
regressor = ["dif_sentiment_bert_value", "returns_stoxx", "returns_ma", "returns_visa", "Volume", "count_all_tweets"]

regressand = ["returns_week"]

df_week_reg = create_df()

reg_predictions, reg_model = regression(df_week_reg, regressor, regressand)

                            OLS Regression Results                            
Dep. Variable:           returns_week   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.759
Method:                 Least Squares   F-statistic:                     123.1
Date:                Wed, 07 Dec 2022   Prob (F-statistic):           1.73e-68
Time:                        22:05:53   Log-Likelihood:                 308.42
No. Observations:                 234   AIC:                            -602.8
Df Residuals:                     227   BIC:                            -578.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [104]:
reg_model.pvalues

const                       9.494638e-01
dif_sentiment_bert_value    9.919889e-20
returns_stoxx               2.729299e-03
returns_ma                  6.182534e-01
returns_visa                4.437239e-01
Volume                      1.605783e-02
count_all_tweets            1.276459e-03
dtype: float64

In [105]:
reset = dg.linear_reset(reg_model, power=3, test_type="fitted", use_f=True)
reset.pvalue

/Users/tobias/Dev/FOM/Master_Thesis/master_thesis_3.9.10/.venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


1.7712849938118904e-15

<p>________________________________________________________________________________</p>
<p>Multiple lineare Regression mit VADER auf alle Tweets</p>

In [106]:
regressor = ["dif_sentiment_vader_value", "returns_stoxx", "returns_ma", "returns_visa", "Volume", "count_all_tweets"]

regressand = ["returns_week"]

df_week_reg = create_df()

reg_predictions, reg_model = regression(df_week_reg, regressor, regressand)

                            OLS Regression Results                            
Dep. Variable:           returns_week   R-squared:                       0.718
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                     96.38
Date:                Wed, 07 Dec 2022   Prob (F-statistic):           1.31e-59
Time:                        22:06:10   Log-Likelihood:                 287.21
No. Observations:                 234   AIC:                            -560.4
Df Residuals:                     227   BIC:                            -536.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [107]:
reg_model.pvalues

const                        8.040902e-01
dif_sentiment_vader_value    1.140359e-10
returns_stoxx                8.142642e-03
returns_ma                   4.995541e-01
returns_visa                 6.324005e-01
Volume                       6.032766e-04
count_all_tweets             2.022125e-04
dtype: float64

In [108]:
reset = dg.linear_reset(reg_model, power=3, test_type="fitted", use_f=True)
reset.pvalue

/Users/tobias/Dev/FOM/Master_Thesis/master_thesis_3.9.10/.venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


9.165383251524327e-26

<p>________________________________________________________________________________</p>
<p>Grundanalyse</p>

In [ ]:
regressor = ["dif_sentiment_bert_value", "returns_stoxx", "returns_ma", "returns_visa", "Volume", "count_all_tweets"]
regressor = ["dif_sentiment_vader_value", "returns_stoxx", "returns_ma", "returns_visa", "Volume", "count_all_tweets"]

regressand = ["returns_week"]

df_week_reg = create_df()

reg_predictions, reg_model = regression(df_week_reg, regressor, regressand)

In [72]:
reset = dg.linear_reset(reg_model, power=3, test_type="fitted", use_f=True)
print("F", np.round(reset.fvalue, 6))
print("p", np.round(reset.pvalue, 6))

F 66.219162
p 0.0


In [87]:
reset = dg.linear_reset(reg_model, power=3, test_type="fitted", use_f=True)
reset.pvalue

/Users/tobias/Dev/FOM/Master_Thesis/master_thesis_3.9.10/.venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


9.165383251524327e-26